In [102]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
import glob
import re
import copy

In [130]:
shadow = 'shadow30_2'
evaluation_path = "../../get_data/geocast_output/" + shadow + "/"
sigo_files = glob.glob(evaluation_path + "gsigo/*")
sigo_file_list = []
for file in sigo_files:
    sigo_file_list.append(file)

# print('receive log')
# print(sigo_file_list)

In [131]:
shadow = 'shadow30_2'
evaluation_path = "../../get_data/geocast_output/" + shadow + "/"
lsgo_files = glob.glob(evaluation_path + "glsgo/*")
lsgo_file_list = []
for file in lsgo_files:
    lsgo_file_list.append(file)

# print('receive log')
# print(lsgo_file_list)

# seed値リストを登録

In [132]:
seed_index = 2
node_index = 3
seed_list = []
nodenum_list = ['200', '300', '400'] ###############nodenumの形式を変更したらここはベタ打ちで変更を加える
regex = re.compile('\d+')

for path in sigo_file_list:
    for line in path.splitlines():
        match = regex.findall(line)
        if match[seed_index] not in seed_list:
            seed_list.append(match[seed_index])
            
print(seed_list)
print('simulation 試行回数',len(seed_list))

['10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '10010', '10011', '10012', '10013', '10014', '10015', '10016', '10017', '10018', '10019', '10020', '10021', '10022', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10038', '10039', '10040', '10041', '10042', '10043', '10044', '10045', '10046', '10047', '10048', '10049']
simulation 試行回数 49


In [133]:
sigo_dict = {}
check_seed = seed_list[0]
sigo_dict[check_seed] = {}
for path in sigo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        sigo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        sigo_dict[match[seed_index]] = {}
        sigo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新

print(sigo_dict)

{'10001': {'200': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10001nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10001nodenum_300.csv', '400': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10001nodenum_400.csv'}, '10002': {'200': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10002nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10002nodenum_300.csv', '400': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10002nodenum_400.csv'}, '10003': {'200': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10003nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10003nodenum_300.csv', '400': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10003nodenum_400.csv'}, '10004': {'200': '../../get_data/geocast_output/shadow30_2/gsigo\\gsigo-seed_10004nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/gsigo\\

In [134]:
lsgo_dict = {}
check_seed = seed_list[0]
lsgo_dict[check_seed] = {}
for path in lsgo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        lsgo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        lsgo_dict[match[seed_index]] = {}
        lsgo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新
        
print(lsgo_dict)

{'10001': {'200': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10001nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10001nodenum_300.csv', '400': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10001nodenum_400.csv'}, '10002': {'200': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10002nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10002nodenum_300.csv', '400': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10002nodenum_400.csv'}, '10003': {'200': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10003nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10003nodenum_300.csv', '400': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10003nodenum_400.csv'}, '10004': {'200': '../../get_data/geocast_output/shadow30_2/glsgo\\glsgo-seed_10004nodenum_200.csv', '300': '../../get_data/geocast_output/shadow30_2/glsgo\\

In [135]:
# 列ごとの平均を返す関数
def col_average(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
#     np.set_printoptions(precision=3, suppress=True)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [136]:
# 列ごとの平均を返す関数
def col_average_zero_delete(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [137]:
# packet delivery rationの平均を計算 SIGO Geocast
total_pdr_list = []
for seed in seed_list:
    pdr_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        pdr = sigo_df['PDR'].mean()
        pdr_list.append(pdr)
    pdr_list.append(seed)
    total_pdr_list.append(pdr_list)

print(total_pdr_list)
print(col_average(total_pdr_list))

[[0.1529412, 0.5, 0.6, '10001'], [0.3674867, 0.5448276, 0.5025016, '10002'], [0.36, 0.4, 0.7, '10003'], [0.3545455, 0.48076919999999995, 0.6575, '10004'], [0.1545455, 0.3, 0.705, '10005'], [0.2, 0.3, 0.6038514, '10006'], [0.35294119999999995, 0.4, 0.605, '10007'], [0.4029412, 0.3, 0.6, '10008'], [0.5074867000000001, 0.4, 0.5, '10009'], [0.45454550000000005, 0.4, 0.5525, '10010'], [0.1, 0.48, 0.6, '10011'], [0.3074867, 0.2, 0.5475, '10012'], [0.2545455, 0.4, 0.7012805, '10013'], [0.2529412, 0.24482759999999998, 0.7575, '10014'], [0.16, 0.4, 0.6, '10015'], [0.2, 0.2, 0.7575, '10016'], [0.2, 0.4, 0.6550709, '10017'], [0.3545455, 0.48, 0.7, '10018'], [0.25, 0.3, 0.6, '10019'], [0.4, 0.5, 0.6512195000000001, '10020'], [0.2, 0.4, 0.7526316, '10021'], [0.3545455, 0.3031724, 0.6575, '10022'], [0.3, 0.4, 0.6012805, '10023'], [0.3545455, 0.2, 0.4, '10024'], [0.209091, 0.3, 0.7, '10025'], [0.3545455, 0.4, 0.5525, '10026'], [0.3, 0.48076919999999995, 0.8512820999999999, '10027'], [0.1, 0.3551724, 

In [138]:
# packet delivery rationの平均を計算 LSGO Geocast
total_pdr_list = []
for seed in seed_list:
    pdr_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        pdr = lsgo_df['PDR'].mean()
        pdr_list.append(pdr)
    pdr_list.append(seed)
    total_pdr_list.append(pdr_list)

print(total_pdr_list)
print(col_average(total_pdr_list))

[[0.2545455, 0.45517240000000003, 0.6525000000000001, '10001'], [0.2, 0.2551724, 0.3, '10002'], [0.26, 0.3, 0.7, '10003'], [0.2545455, 0.42559679999999994, 0.6, '10004'], [0.3545455, 0.2, 0.5, '10005'], [0.2, 0.4, 0.4525, '10006'], [0.4058824, 0.2, 0.705, '10007'], [0.3545455, 0.3, 0.6, '10008'], [0.1545455, 0.3, 0.6, '10009'], [0.3, 0.3, 0.4525, '10010'], [0.4, 0.38, 0.6475, '10011'], [0.3674867, 0.24482759999999998, 0.6525000000000001, '10012'], [0.2545455, 0.2, 0.4525, '10013'], [0.3058824, 0.3448276, 0.4, '10014'], [0.3129412, 0.4, 0.6, '10015'], [0.2, 0.2359416, 0.505, '10016'], [0.1545455, 0.4448276, 0.7025709, '10017'], [0.31454550000000003, 0.2, 0.40678570000000003, '10018'], [0.31454550000000003, 0.3, 0.605, '10019'], [0.3, 0.4448276, 0.6, '10020'], [0.2, 0.2, 0.5051316, '10021'], [0.45454550000000005, 0.5, 0.5525, '10022'], [0.0, 0.3, 0.4, '10023'], [0.054545500000000004, 0.2, 0.5067857, '10024'], [0.1, 0.24482759999999998, 0.8542856999999999, '10025'], [0.2545455, 0.24482759

In [139]:
# Overheadの平均を計算 SIGO Geocast
total_overhead_list = []
for seed in seed_list:
    overhead_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        mr_recvs = sigo_df['mr_recvs'].sum() #mr内の受信ノード数
        sequence_count = sigo_df[(sigo_df['PDR'] != 0)]['mr_recvs'].count()
        geocast_packets = sigo_df[(sigo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = sigo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        if mr_recvs == 0:
            overhead = 0 #いったん仮置き
        else:
            overhead = packet_count / mr_recvs
        overhead_list.append(overhead)
    overhead_list.append(seed)
    if 0 not in overhead_list:
        total_overhead_list.append(overhead_list)
    else:
        print('seed', seed, 'list', overhead_list)
print(col_average(total_overhead_list))

[1.66513945 1.77041266 1.82778324]


In [140]:
# Overheadの平均を計算 LSGO Geocast
total_overhead_list = []
for seed in seed_list:
    overhead_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        mr_recvs = lsgo_df['mr_recvs'].sum() #mr内の受信ノード数
        sequence_count = lsgo_df[(lsgo_df['PDR'] != 0)]['mr_recvs'].count()
        geocast_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        if mr_recvs == 0:
            overhead = 0 #いったん仮置き
        else:
            overhead = packet_count / mr_recvs
        overhead_list.append(overhead)
    overhead_list.append(seed)
    if 0 not in overhead_list:
        total_overhead_list.append(overhead_list)
    else:
        print('seed', seed, 'list', overhead_list)
print(col_average(total_overhead_list))

seed 10023 list [0, 2.2688172043010755, 2.069364161849711, '10023']
[3.27698037 2.48952306 2.38258675]


In [141]:
# packet数の平均を計算 SIGO Geocast
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        geocast_packets = sigo_df[(sigo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = sigo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        packet_list.append(packet_count)
    packet_list.append(seed)
#     print(packet_list)
    total_packet_list.append(packet_list)
print(col_average(total_packet_list))

[ 89.3877551  204.63265306 478.26530612]


In [142]:
# packet数の平均を計算 LSGO Geocast
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        geocast_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        packet_list.append(packet_count)
    packet_list.append(seed)
#     print(packet_list)
    total_packet_list.append(packet_list)
print(col_average(total_packet_list))

[159.59183673 251.3877551  553.65306122]


In [145]:
# geocast パケット数
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = sigo_dict[seed][node]
        sigo_df = pd.read_csv(path)
        geocast_packets = sigo_df[(sigo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = sigo_df[(sigo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        packet_list.append(geocast_packets)
    packet_list.append(seed)
    total_packet_list.append(packet_list)
print('sigo geocast packet average' , col_average(total_packet_list))

sigo geocast packet average [3.02040816 3.7755102  7.06122449]


In [146]:
total_packet_list = []
for seed in seed_list:
    packet_list = []   # index 0: 200nodes  index 1: 300nodes index:2 400nodes
    for node in nodenum_list:
        path = lsgo_dict[seed][node]
        lsgo_df = pd.read_csv(path)
        geocast_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['geocast_packets'].sum()
        flooding_packets = lsgo_df[(lsgo_df['PDR'] != 0)]['flooding_packets'].sum()
        packet_count = geocast_packets + flooding_packets
        packet_list.append(geocast_packets)
    packet_list.append(seed)
    total_packet_list.append(packet_list)
print('lsgo geocast packet average' , col_average(total_packet_list))

lsgo geocast packet average [ 81.42857143  75.40816327 135.        ]


In [116]:
test_path = sigo_dict['10001']['200']
test_df = pd.read_csv(test_path)
print(test_df['mr_recvs'].sum()) #mr_recvsの合計

#pdr が 0ではないmr_recvの合計
print(test_df[(test_df['PDR'] != 0)]['mr_recvs'].sum()) 

#pdrが0ではない　もの(packet)だけをカウント
print(test_df[(test_df['PDR'] != 0)]['mr_recvs'].count())

94
94
6
